In [57]:
# import libraries 
import pandas as pd
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os 
import zipfile

In [58]:
### Unzip data if needed 



# # Define paths for each zip file
# zip_files = {
#     "dataset": "FullDataset/bondugula_JDO_20230125_SLIM.csv.zip",
#     "notebook1": "FullDataset/full-data-nb.ipynb.zip",
#     "notebook2": "FullDataset/full-data-practice-041223.ipynb.zip",
#     "notebook3": "FullDataset/full-data-practice-061023.ipynb.zip",
#     "notebook4": "FullDataset/full-data-practice-20230225.ipynb.zip",
#     "unknown_csv": "FullDataset/s8-acetyl+sirt-output.csv.zip"
# }

# # Function to unzip files
# def unzip_file(zip_path, extract_to):
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)
#     print(f"Extracted {zip_path} to {extract_to}")

# # Directory to extract all files
# extract_dir = "Unzipped_Files"

# # Create the extraction directory if it doesn't exist
# os.makedirs(extract_dir, exist_ok=True)

# # Unzip each file
# for name, path in zip_files.items():
#     output_folder = os.path.join(extract_dir, name)  # Folder based on each file's label
#     os.makedirs(output_folder, exist_ok=True)
#     unzip_file(path, output_folder)

# View dataset

In [59]:
# Define path to data and define it
file_path = "FullDataset/unzipped_data/dataset/bondugula_JDO_20230125_SLIM.csv"
data = pd.read_csv(file_path, index_col=0)

# View data
print(data.columns)
data.head(5)

Index(['Residue', 'E6', 'E20', 'Protein', 'No.', 'Res', 'isUnstruct', 'E6.1',
       'E20.1', 'E22', 'Vkbat', 'chou_fasman', 'sspro_5', 'gor4', 'dsc',
       'jnet', 'psipred', '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U',
       'ProteinID'],
      dtype='object')


,Residue,E6,E20,Protein,No.,Res,isUnstruct,E6.1,E20.1,E22,...,gor4,dsc,jnet,psipred,# homologues,HAS_H,HAS_S,HAS_O,HAS_U,ProteinID
0,D,0.926212,0.926212,2BDE,1,D,0.954544,NaN,NaN,NaN,...,Other,Other,Other,Other,6,0,0,0,1,2BDE_0
1,T,1.307625,2.024379,2BDE,2,T,0.805483,NaN,NaN,NaN,...,Other,Other,Other,Other,6,0,0,0,1,2BDE_0
2,H,1.321518,1.668090,2BDE,3,H,0.654102,NaN,NaN,NaN,...,Other,Other,Other,Helix,6,0,0,0,1,2BDE_0
3,K,0.528085,0.845275,2BDE,4,K,0.501600,NaN,NaN,NaN,...,Other,Other,Other,Helix,6,0,0,1,1,2BDE_0
4,V,0.000000,0.583619,2BDE,5,V,0.280561,NaN,NaN,NaN,...,Sheet,Sheet,Sheet,Helix,6,0,1,0,1,2BDE_0


We know the descriptors are:
* E6 (?)
* E20 (?)
* isUnstruct (Disorder Propensity)

The target that we need to define is the switch. This occurs when HAS_H, HAS_S, HAS_O, HAS_U added together is greater than 1. We need to make this column

In [60]:
# Create a copy dataframe of region of interest 
switch_det = data.iloc[:, -5:-1]

# Define new column that detects the switch occuring 
switch_det['switch'] = (switch_det.sum(axis=1) > 1).astype(int) # Create binar column, True = 1 and False = 0

# Print values
switch_det.head(3)

,HAS_H,HAS_S,HAS_O,HAS_U,switch
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0


In [61]:
print(switch_det['switch'].value_counts())

switch
0    914237
1    179049
Name: count, dtype: int64


In [62]:
# Add our switch column to the main one 
data2 = data.copy()

data2 = pd.concat([data2, switch_det['switch']], axis=1)
data2.head(5)

,Residue,E6,E20,Protein,No.,Res,isUnstruct,E6.1,E20.1,E22,...,dsc,jnet,psipred,# homologues,HAS_H,HAS_S,HAS_O,HAS_U,ProteinID,switch
0,D,0.926212,0.926212,2BDE,1,D,0.954544,NaN,NaN,NaN,...,Other,Other,Other,6,0,0,0,1,2BDE_0,0
1,T,1.307625,2.024379,2BDE,2,T,0.805483,NaN,NaN,NaN,...,Other,Other,Other,6,0,0,0,1,2BDE_0,0
2,H,1.321518,1.668090,2BDE,3,H,0.654102,NaN,NaN,NaN,...,Other,Other,Helix,6,0,0,0,1,2BDE_0,0
3,K,0.528085,0.845275,2BDE,4,K,0.501600,NaN,NaN,NaN,...,Other,Other,Helix,6,0,0,1,1,2BDE_0,1
4,V,0.000000,0.583619,2BDE,5,V,0.280561,NaN,NaN,NaN,...,Sheet,Sheet,Helix,6,0,1,0,1,2BDE_0,1


# Data Pre-Processing

## Lets check our columns have valid data

In [63]:
# See if there is any missing values 
print("Missing data:")
print(data2.isnull().sum())

Missing data:
Residue               0
E6                 3710
E20                3710
Protein               0
No.                   0
Res                   0
isUnstruct            0
E6.1            1093286
E20.1           1093286
E22             1093286
Vkbat                 0
chou_fasman           0
sspro_5               0
gor4                  0
dsc                   0
jnet                  0
psipred               0
# homologues          0
HAS_H                 0
HAS_S                 0
HAS_O                 0
HAS_U                 0
ProteinID             0
switch                0
dtype: int64


Based off Jonathan's notebook E6.1, E20.1, E22 are not used at all. We can drop these. As far as the 3710 entries missing we can drop those rows as well.

In [64]:
# Drop those missing data values
data2.drop(columns=['E6.1', 'E20.1', 'E22'], inplace=True)
data2.dropna(inplace=True)

# Normalize data 

In [65]:
# View data descriptive stats
data2.describe()

,E6,E20,No.,isUnstruct,Vkbat,# homologues,HAS_H,HAS_S,HAS_O,HAS_U,switch
count,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06,1.089576e+06
mean,4.613143e-01,7.981048e-01,1.800823e+02,2.363689e-01,2.752236e+00,1.674654e+01,4.094391e-01,2.390545e-01,2.641165e-01,2.677060e-01,1.638463e-01
std,6.268500e-01,9.191382e-01,1.657662e+02,2.300717e-01,1.629132e+00,3.857491e+01,4.917306e-01,4.265063e-01,4.408618e-01,4.427637e-01,3.701362e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,5.706888e-04,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.083201e-02,6.200000e+01,6.579392e-02,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.079314e-02,3.989436e-01,1.340000e+02,1.516327e-01,2.400000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.515738e-01,1.383933e+00,2.480000e+02,3.318337e-01,3.000000e+00,1.500000e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
max,2.581541e+00,4.207811e+00,1.505000e+03,1.000000e+00,9.000000e+00,1.174000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


As expected all different scales. We need to scale the data so its on a normal distribution. We also need to One Hot encode all the categorical data

In [66]:
# Initialize Standard Scaler 
scaler = StandardScaler()

# Seperate features from target 
data_quantitative = data2[["E6", 'E20', "isUnstruct", "Vkbat"]]

# Fit and transform data 
standardized_data = scaler.fit_transform(data_quantitative)

# Convert to dataframe
standardized_df = pd.DataFrame(standardized_data, columns=data_quantitative.columns, index=data2.index)
standardized_df.head(3)

,E6,E20,isUnstruct,Vkbat
0,0.741642,0.139378,3.121528,-1.075564
1,1.350102,1.334157,2.473639,-0.216211
2,1.372264,0.946523,1.815667,1.072820


## One Hot Encode categorical data (Qualitative features)

The categorical data that Jonathan used were 'chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred'

# Modeling Quantitative Only

In [70]:
quantitative = pd.concat([standardized_df, data2['switch']], axis=1)
quantitative.head(3)

,E6,E20,isUnstruct,Vkbat,switch
0,0.741642,0.139378,3.121528,-1.075564,0
1,1.350102,1.334157,2.473639,-0.216211,0
2,1.372264,0.946523,1.815667,1.072820,0


## Logit Classifier

In [73]:
# Now lets define features and target 
X = quantitative.iloc[:, :-1]
y = quantitative.iloc[:, -1]

# Add intercept to the model
X = sm.add_constant(X)

# Create Train test split based on features and target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42, shuffle=True)

# Create Logistic model
model = sm.Logit(y_train, X_train)
result = model.fit()

# Review summary of training
print(result.summary())
print()

Optimization terminated successfully.
         Current function value: 0.442336
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 switch   No. Observations:               871660
Model:                          Logit   Df Residuals:                   871655
Method:                           MLE   Df Model:                            4
Date:                Sun, 10 Nov 2024   Pseudo R-squ.:                0.008012
Time:                        22:27:44   Log-Likelihood:            -3.8557e+05
converged:                       True   LL-Null:                   -3.8868e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6466      0.003   -561.220      0.000      -1.652      -1.641
E6             0.1286      0.

In [75]:
# Predict probabilities for the test data and train data
y_pred_prob_test = result.predict(X_test)
y_pred_prob_train = result.predict(X_train)

# Convert probabilities to binary predictions (0 or 1)
y_pred_test = (y_pred_prob_test > 0.5).astype(int)  # Using 0.5 as the threshold for now
y_pred_train = (y_pred_prob_train > 0.5).astype(int)

# Compare predicted values to actual values
acc_test = accuracy_score(y_test, y_pred_test)
acc_train = accuracy_score(y_train, y_pred_train)

print(f"Accuracy Train: {acc_train:.4f}")
print(f"Accuracy Test: {acc_test:.4f}")

Accuracy Train: 0.8362
Accuracy Test: 0.8359


We can see how the model is not overfitting the data at all. The train and testing sets are nearly identical performances. Let see if an OLS model can do better.

## OLS Model

In [22]:
# Create OLS model 
ols = sm.OLS(y_train, X_train)

# Obtain result
ols_result = ols.fit()

# Print summary 
print(ols_result.summary())
print()

                            OLS Regression Results                            
Dep. Variable:                 switch   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1549.
Date:                Sun, 10 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:11:39   Log-Likelihood:            -3.6807e+05
No. Observations:              871660   AIC:                         7.362e+05
Df Residuals:                  871656   BIC:                         7.362e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1393      0.001    212.346      0.0